In [1]:
%pip install pandas
%pip install Faker

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import zipfile
import pandas as pd
from pathlib import Path
import urllib.request

SEED = 42

def load_kaggle_data():
    file_path = Path("./datasets/archive.zip")
    if not file_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://www.kaggle.com/api/v1/datasets/download/amirmahdiabbootalebi/salary-by-job-title-and-country"
        urllib.request.urlretrieve(url, file_path)
    zf = zipfile.ZipFile("./datasets/archive.zip")

    return pd.read_csv(zf.open('Salary.csv'))

kaggle_data = load_kaggle_data ()
print (*kaggle_data.columns, sep='\n')

Age
Gender
Education Level
Job Title
Years of Experience
Salary
Country
Race
Senior


In [30]:
from faker import Faker
import random
import urllib.request
import json
import numpy as np

# Copy dataset and only take the columns we need
dataset_clean = kaggle_data[["Salary", "Country", "Job Title"]]

# Rename the columns that were copied
dataset_clean = dataset_clean.rename (columns={"Salary": "salary", "Country": "work_location", "Job Title": "job_role"})

Male images: (2515,)
Female images: (2485,)


In [ ]:
def getFromUrl (url):
    with urllib.request.urlopen(url) as response:
        # Read the response data
        data = response.read().decode('utf-8')
        
        # Parse the JSON data
        json_data = json.loads(data)
        
        return json_data

user_images = np.array (getFromUrl (f"https://randomuser.me/api/?inc=picture&results=5000&seed={SEED}")["results"])

male_urls = []
female_urls = []

# Extract and categorize the URLs
for user in user_images:
    thumbnail_url = user["picture"]["thumbnail"]
    if "women" in thumbnail_url or "female" in thumbnail_url:
        female_urls.append(thumbnail_url)
    elif "men" in thumbnail_url or "male" in thumbnail_url:
        male_urls.append(thumbnail_url)

# Convert lists to NumPy arrays
user_images_male = np.array(male_urls)
user_images_female = np.array(female_urls)

print(f"Male images: {user_images_male.shape}")
print(f"Female images: {user_images_female.shape}")

np.save ("./datasets/user_images_male.npy", user_images_male)
np.save ("./datasets/user_images_female.npy", user_images_female)

In [32]:
# Create mock data for the remaininig columns
Faker.seed(SEED)
faker = Faker()
random.seed (SEED)

def generateGender ():
    rng = random.random()

    if rng < 0.50:
        return "m"
    else:
        return "f"
    
def generateNameForGender (gender):
    if gender == "m":
        return faker.name_male ()
    elif gender == "f":
        return faker.name_female ()
    else:
        raise ValueError("Invalid gender provided: " + str(gender))
    
def generatePicForGender (gender):
    if gender == "m":
        return np.random.choice (user_images_male)
    elif gender == "f":
        return np.random.choice (user_images_female)
    else:
        raise ValueError("Invalid gender provided: " + str(gender))
    
def generateReportsTo ():
    return random.randint (1, len (dataset_clean))

In [33]:
dataset_clean["id"] = [i for i in range (len (dataset_clean))]
dataset_clean["gender"] = [generateGender () for i in range (len (dataset_clean))]
dataset_clean["name"] = dataset_clean["gender"].apply(generateNameForGender)
dataset_clean["pfp"] = dataset_clean["gender"].apply(generatePicForGender)
dataset_clean["reports_to"] = [generateReportsTo () for i in range (len (dataset_clean))]
dataset_clean["phone_number"] = [f"{random.randint(200, 999):03d}{random.randint(200, 999):03d}{random.randint(200, 999):04d}" for i in range (len (dataset_clean))]

print (*dataset_clean.columns, sep="\n")
print (dataset_clean.head ())

salary
work_location
job_role
gender
name
pfp
phone_number
id
reports_to
     salary work_location           job_role gender               name  \
0   90000.0            UK  Software Engineer      f       Allison Hill   
1   65000.0           USA       Data Analyst      m        Noah Rhodes   
2  150000.0        Canada            Manager      m  Brandon Henderson   
3   60000.0           USA    Sales Associate      m      Daniel Wagner   
4  200000.0           USA           Director      f   Christina Santos   

                                                 pfp phone_number  id  \
0  https://randomuser.me/api/portraits/thumb/wome...   6344180831   0   
1  https://randomuser.me/api/portraits/thumb/men/...   7733950591   1   
2  https://randomuser.me/api/portraits/thumb/men/...   5833830965   2   
3  https://randomuser.me/api/portraits/thumb/men/...   2104000652   3   
4  https://randomuser.me/api/portraits/thumb/wome...   5697380399   4   

   reports_to  
0        3798  
1        55

In [ ]:
import subprocess

# Save the data to the disk
FILENAME = "./datasets/dataset_clean"
dataset_clean.to_parquet (FILENAME + ".parquet")
dataset_clean.to_json (FILENAME + ".json", orient='records', indent=4)

# Save the data to the database
# try:
#     MONGO_URI = "mongodb://localhost:27017/employeeDB"
#     MONGO_COLLECTION = "employees"
#     result = subprocess.run(["mongoimport", "--uri", MONGO_URI, "--collection", MONGO_COLLECTION, "--drop", "--file", FILENAME + ".json", "--jsonArray"], capture_output=True, text=True, check=True)

#     print("Standard Output:")
#     print(result.stdout)
#     print("Standard Error:")
#     print(result.stderr)
# except subprocess.CalledProcessError as e:
#     print(f"Error executing command: {e}")
#     print(f"Return Code: {e.returncode}")
#     print(f"Output: {e.output}")
#     print(f"Error Output: {e.stderr}")

Standard Output:

Standard Error:
2025-07-10T09:34:43.302-0400	connected to: mongodb://localhost:27017/employeeDB
2025-07-10T09:34:43.315-0400	dropping: employeeDB.employees
2025-07-10T09:34:43.571-0400	6684 document(s) imported successfully. 0 document(s) failed to import.

